# Introduction to tree sequences for SLiM

In [ ]:
import tskit, pyslim, msprime
%load_ext slim_magic

**Note:** below we *set the seed* several times,
so the output of this notebook is consistent.
However, you almost certainly *don't* want to do that in your own scripts.

## Recapitation

In this short introduction to how recapitation, we'll:

1. run a *very* small simulation in SLiM;
2. look at the resulting tree sequence to see why it needs recapitating;
3. recapitate, with pyslim and check that it worked;
4. add neutral mutations, and
5. output to VCF.

Here is a small SLiM script, that simulates a population of diploid size 5 for 3 generations with a WF model.
The genome has $10^7$bp and the recombination rate is $10^{-8}$, and there are no mutations.

In [ ]:
%%slim_ts --out ts
initialize() {
   initializeTreeSeq();
   initializeMutationRate(0.0);
   initializeMutationType("m1", 0.5, "f", 0.0);
   initializeGenomicElementType("g1", m1, 1.0);
   initializeGenomicElement(g1, 0, 9999999);
   initializeRecombinationRate(1e-8);
   setSeed(123);
}

1 early() {
   sim.addSubpop("p1", 5);
}

3 late() {
   sim.treeSeqOutput("tmp.trees");
}

Here's statistics about the resulting tree sequence.
On the command line we could see this by doing `tskit info tmp.trees`.

In [ ]:
ts

I made this a very small simulation so we could actually look at a picture of the whole thing.
Here it is! The gray region shows the relationships between all 10 sampled genomes over the first region of genome;
the white shows this on the second region (thus, the "two trees" listed above).
However, note that each "tree" is in fact a bunch of trees - the genealogies have not *coalesced*.

In [ ]:
ts.draw_svg(size=(800,300), y_axis=True)

The simulation *has* coalesced if each tree has a single root. Here's the easy way to check this.

In [ ]:
max_num_roots = max([t.num_roots for t in ts.trees()])
print(f"Maximum number of roots: {max_num_roots}")

## Recapitation

Now, we "recapitate"!
What this means is that we run a coalescent simulation to simulate the ancestry from the oldest bit of the tree sequence.
It uses `msprime`, and here we're asking for an ancestral effective population size of 2;
so this is roughly as if we'd had a randomly mating population of diploid size 2 for a while before the start of the SLiM simulation.
(I chose such a small size so we'd still be able to plot the whole history.)

In [ ]:
ts = pyslim.recapitate(ts, ancestral_Ne=2, recombination_rate=1e-8, random_seed=12)

Read the warning, and do think about time scales, but we won't worry about it here.

Now, we see that all trees *have* got a single root,
i.e., the tree sequence is coalesced.

In [ ]:
max_num_roots = max([t.num_roots for t in ts.trees()])
print(f"Maximum number of roots: {max_num_roots}")

In [ ]:
ts.draw_svg(size=(800,400), y_axis=True)

# Mutations

We simulated in SLiM with *zero* mutations;
this makes sense because if all mutations are neutral,
we might as well just put them down on the tree sequence afterwards.
Here's how to do that with `msprime`;
we could choose any number of mutation models,
but I'm doing the "SLiM mutation model"
because that's what we'd need to use if we wanted to load the simulation back into SLiM later.

In [ ]:
ts = msprime.sim_mutations(
    ts,
    model=msprime.SLiMMutationModel(type=0, next_id=pyslim.next_slim_mutation_id(ts)),
    rate=1e-8,
    random_seed=12345,
)

Now, the tree sequence has 10 mutations.

In [ ]:
ts

They are shown as red "x"s on the plot.

In [ ]:
ts.draw_svg(size=(800,400))

Here are the genotypes, coded as `0`=ancestral and `1`=derived;
there are no sites with more than mutation (but there could be!).

In [ ]:
ts.genotype_matrix()

However, writing out to VCF has some... weird REF and ALT columns.

In [ ]:
with open("temp.vcf", "w") as f:
    ts.write_vcf(f)

In [ ]:
%%bash
cat temp.vcf

That's because of some techinical details about the underlying mutation model in SLiM.
To get a VCF with actual nucleotides in REF and ALT,
you can use pyslim to randomly generate some.

In [ ]:
nts = pyslim.convert_alleles(pyslim.generate_nucleotides(ts))

Now, mutations have nucleotides as `derived_state`
(plus, check out all the other information there!):

In [ ]:
nts.mutation(0)

Here's the 10 haplotypes (*note! don't use this code for bigger tree sequences!*):

In [ ]:
list(nts.haplotypes())  # DO NOT DO THIS FOR BIGGER TREE SEQUENCES

In [ ]:
with open("temp.vcf", "w") as f:
    nts.write_vcf(f)

In [ ]:
%%bash
cat temp.vcf

Ah, that looks much better (and more parseable by other software).

## The tables

In [ ]:
t = nts.tables

In [ ]:
t.individuals

In [ ]:
t.nodes

In [ ]:
t.edges

In [ ]:
t.sites

In [ ]:
t.mutations